In [1]:
import requests
import pandas as pd

import re
import os
from bs4 import BeautifulSoup

In [2]:
SESSION = requests.Session()
URL = "http://ditich.vn/FrontEnd/DiTich"

In [3]:
response = SESSION.get(url=URL, timeout=5)
html = response.content
        
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [4]:
totalrecord = soup.find(lambda tag: tag.name=="span" and "Tổng số" in tag.text).text.split(':')[1].strip()
totalrecord = int(totalrecord)

In [5]:
PARAMS = {
    "cpage": 1,
    "rpage": totalrecord
}

response = SESSION.get(url=URL, params=PARAMS, timeout=5)
html = response.content
        
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
a_tags = soup.find_all("a", title="Xem chi tiết")

In [6]:
loclist = []
MAIN_URL = "http://ditich.vn"

for a_tag in a_tags:
    thisloc = a_tag.find("span", string=True)
    if not thisloc: print(a_tag.find("h2", string=True).text + " has no loc")
    else: thisloc = thisloc.text
    loclist.append({
        "name": a_tag.find("h2", string=True).text,
        "url": MAIN_URL + a_tag['href'],
        "img": MAIN_URL + a_tag.find("img")['src'],
        "loc": thisloc
    })

Chùa Thắng Hạ has no loc
Từ đường họ Lê Văn has no loc


In [7]:
len(loclist)

4024

In [8]:
loclist[2227]

{'name': 'Khu lưu niệm đồng chí Nguyễn Văn Cừ',
 'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=3883',
 'img': 'http://ditich.vn\\upload\\images\\ditich\\LN-N0005.jpg',
 'loc': 'Tỉnh Bắc Ninh,huyện Từ Sơn, xã Phù Khê'}

In [9]:
def getLocationInfo(location):
    response = SESSION.get(url=location['url'])
    soup = BeautifulSoup(response.content, "html.parser")

    divs = soup.find_all("div", class_="hl__illustrated-list__list-item") 

    for div in divs:
        span = div.find(lambda tag: tag.name=="span" and ":" in tag.text)
        
        if not span:
            continue
        
        span = span.text.split(':')
        key = span[0].strip()
        val = span[1].strip()
        if len(key) > 0 and len(val) > 0:
            location[key] = val
        else:
            print("ERR in split : for span" + span[0])

In [10]:
from tqdm import tqdm

In [11]:
for loc in tqdm(loclist):
    getLocationInfo(loc)

100%|██████████████████████████████████████████████████████████████████████████████| 4024/4024 [11:09<00:00,  6.01it/s]


In [12]:
if not os.path.isfile('DiTich_ALT2.csv'):
    df = pd.DataFrame(loclist)
    df.to_csv("DiTich_ALT2.csv", index=False)
else:
    df = pd.read_csv("DiTich_ALT.csv")
df

,name,url,img,loc,Loại hình di tích,Loại hình xếp hạng,Xếp hạng,Niên đại tuyệt đối,Niên đại chủ đạo,Tọa độ,Đối tượng thờ,Dấu vết vật chất sớm nhất,Quyết định xếp hạng,Tên khác,Niên đại khởi dựng
0,Đền Đậu An,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\De-D0129...,"Hưng Yên, Tiên Lữ, An Viên",Đền,Di tích kiến trúc nghệ thuật,Di tích cấp Quốc gia,Đền chính,Đền chính,"20°40'47.7""N 106°06'22.9""E","Ngọc Hoàng, Tiên Thiên, Địa Thiên",+01 Tháp đất nung,năm 1989,NaN,NaN
1,Chùa Hoàng Hà (Kim Sơn tự),http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Ch-H0162...,"Hưng Yên, Tiên Lữ, Trung Dũng",Chùa,Di tích kiến trúc nghệ thuật,NaN,Thượng điện,Tam bảo,"20°40'54.7""N 106°09'35.7""E",Phật,Dấu vết trên di vật,NaN,NaN,NaN
2,Chùa Phú Hoà,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Ch-P0158...,"Hưng Yên, Khoái Châu, Bình Kiều",Chùa,Di tích kiến trúc nghệ thuật,NaN,NaN,Nhà mẫu,"20°49'14.0""N 105°58'12.9""E",NaN,Dấu vết trên di vật,NaN,NaN,NaN
3,Miếu Sơn Thần,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Mi-S0004...,"Vĩnh Phúc, Tam Đảo, Đại Đình",Miếu,NaN,NaN,NaN,NaN,"21°27'44.2""N 105°35'04.7""E",thần,NaN,NaN,NaN,NaN
4,Đền Hạ Tụ Nghĩa,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\De-H0058...,"Vĩnh Phúc, Tam Đảo, Đại Đình",Đền,NaN,NaN,NaN,NaN,21°28’53’’B; 105°36’47’’Đ,thần,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,Lăng đá hiệp Hòa,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\LM-H0003...,"Tỉnh Bắc Giang, huyện Hiệp hòa",Lăng Mộ,NaN,Di tích cấp Quốc gia,NaN,NaN,"21°20'40""N 105°58'43""E","Quận công La Đoan Trực, Quận công Ngọ Công Quế",thế kỷ 17,Số 29VH-QĐ ngày 1311964,"Lăng Linh Dương, lăng họ Ngọ",Thời Lê
4020,Chùa Cổ Linh,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Ch-C0033...,"Thành Phố Hà Nội, Huyện Thanh Trì, xã Lĩnh Nam",Chùa,Di tích kiến trúc nghệ thuật,Di tích cấp Quốc gia,NaN,NaN,"20°58'38.1""N 105°53'47.1""E",Phật,NaN,ngày 13.3.1994.,Cổ Linh Tự,NaN
4021,Chùa Cầu Đông,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Ch-C0015...,"Thành phố Hà Nội, quận Hoàn Kiếm, phường Hàng Đào",Chùa,Di tích kiến trúc nghệ thuật,Di tích cấp Quốc gia,NaN,NaN,"21°02'10""N 105°50'59""E",thờ Phật và thờ Thái sư Trần Thủ Độ va vợ ông,NaN,số 1570-VHQĐ ngày 591989,Đông Môn Tự,TK 17
4022,Chùa Bộc,http://ditich.vn/FrontEnd/DiTich/Form?do=&Item...,http://ditich.vn\upload\images\ditich\Ch-B0031...,"Thành phố Hà Nội, quận Đống Đa, phường Quang T...",Chùa,Di tích lịch sử văn hóa,Di tích cấp Quốc gia,NaN,NaN,"21°02'09""N 105°49'20""E","Phật, Vua Quang Trung",NaN,Số 29-VH-QĐ ngày 13.1.1964,Sùng Phúc Tự hay Thiên Phúc Tự,1676


In [9]:
%load_ext line_profiler 

In [12]:
%lprun -f getLocationInfo getLocationInfo(loclist[2227])

Timer unit: 1e-07 s

Total time: 0.547742 s
File: C:\Users\Legion\AppData\Local\Temp\ipykernel_21008\1067590679.py
Function: getLocationInfo at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def getLocationInfo(location):
     2         1    4905878.0 4905878.0     89.6      response = SESSION.get(url=location['url'])
     3         1     499225.0 499225.0      9.1      soup = BeautifulSoup(response.content, "html.parser")
     4                                           
     5         1      62990.0  62990.0      1.1      divs = soup.find_all("div", class_="hl__illustrated-list__list-item") 
     6                                           
     7         9         26.0      2.9      0.0      for div in divs:
     8         9       8925.0    991.7      0.2          span = div.find(lambda tag: tag.name=="span" and ":" in tag.text)
     9                                                   
    10         6         

In [17]:
loclist

[{'name': 'Đền Đậu An',
  'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=6193',
  'img': 'http://ditich.vn\\upload\\images\\ditich\\De-D0129.jpg',
  'loc': 'Hưng Yên, Tiên Lữ, An Viên'},
 {'name': 'Chùa Hoàng Hà (Kim Sơn tự)',
  'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=6192',
  'img': 'http://ditich.vn\\upload\\images\\ditich\\Ch-H0162.jpg',
  'loc': 'Hưng Yên, Tiên Lữ, Trung Dũng'},
 {'name': 'Chùa Phú Hoà',
  'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=6191',
  'img': 'http://ditich.vn\\upload\\images\\ditich\\Ch-P0158.jpg',
  'loc': 'Hưng Yên, Khoái Châu, Bình Kiều'},
 {'name': 'Miếu Sơn Thần',
  'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=5733',
  'img': 'http://ditich.vn\\upload\\images\\ditich\\Mi-S0004.jpg',
  'loc': 'Vĩnh Phúc, Tam Đảo, Đại Đình'},
 {'name': 'Đền Hạ Tụ Nghĩa',
  'url': 'http://ditich.vn/FrontEnd/DiTich/Form?do=&ItemId=5732',
  'img': 'http://ditich.vn\\upload\\images\\ditich\\De-H0058.jpg',
  'loc': 'Vĩnh Phúc